# 1. Thư viện

In [1]:
import os
import glob
import json
from dotenv import load_dotenv

from langchain_community.document_loaders import Docx2txtLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_qdrant import Qdrant

d:\Download_Python\lib\importlib\__init__.py:127: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


# 2. Thực thi

## 2.1. Load folder .docx

In [2]:
folder_data_docx = r"data\trich_dan_luat\docx\Luat"
os.chdir(folder_data_docx)

## 2.2. Load file .json

In [3]:
file_name_json = r'D:\Download_Github_Desktop\Law_ChatBot\Vector_Database_Qdrant\data\trich_dan_luat\json\extract_Data_FileName.json'
with open(file_name_json, 'r', encoding='utf-8') as file:
    data_json = json.load(file)

## 2.3. Model Embedidng

In [4]:
EMBEDDINGS_MODEL = "dangvantuan/vietnamese-embedding"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDINGS_MODEL,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

## 2.4. Semantic Chunker

In [5]:
text_splitter = SemanticChunker(
    embeddings=embeddings, breakpoint_threshold_type="gradient"
)

## 2.5. Upload files

In [6]:
load_dotenv()

URL_QDRANT = os.getenv("URL_QDRANT")
API_QDRANT = os.getenv("API_QDRANT")

url = URL_QDRANT
api_key = API_QDRANT

In [7]:
for idx, f in enumerate(glob.glob('*.docx')):
	raw_documents = Docx2txtLoader(f).load()
	docs = text_splitter.split_documents(raw_documents)
	metadata = data_json[idx]
	for doc in docs:
		setattr(doc, "metadata", metadata) 
	qdrant = Qdrant.from_documents(
		documents=docs,
		embedding=embeddings,
		url=url,
		api_key = api_key,
		collection_name="semantic_Luat_dangvantuan",
		metadata_payload_key="metadata"
	)

IndexError: index out of range in self